In [1]:
import json
import numpy as np
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences
import json
import keras
import pandas as pd
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Dropout, Activation, Embedding
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

import json
from keras.models import model_from_json

# Import libraries
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
# nltk.download()
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

import string
import re
from keras.preprocessing.text import Tokenizer
from nltk.stem import WordNetLemmatizer 

In [2]:
# we're still going to use a Tokenizer here, but we don't need to fit it
tokenizer = Tokenizer(num_words=10000)
# for human-friendly printing
labels = ['fake', 'real']

# read in our saved dictionary
with open('./Dictionary_Models/glove_models_dictionary.json', 'r') as dictionary_file:
    dictionary = json.load(dictionary_file)

In [3]:
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text)
#     words = pad_sequences(words, padding='post', maxlen=23)
#     print(text)
    wordIndices = []
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
    return wordIndices

In [4]:
json_file = open('./NN_Models/Glove_BLSTM_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)

model.load_weights('./NN_Models/Glove_BLSTM_model.h5')


In [5]:
tweetText = 'there is a fire at melville mountain'

tweetText = convert_text_to_index_array(tweetText)

input = pad_sequences([tweetText], padding='post', maxlen=100)
    
pred = model.predict_generator(input)

def rounding(results):
    '''Results needs to be rounded to 0 or 1 for fake or real, respectively'''
    if results < 0.5:
        return 0
    else:
        return 1
    
predictions_final = [rounding(x) for x in pred]

predictions_final

Instructions for updating:
Please use Model.predict, which supports generators.


[1]

# Connecting to Tweepy

In [6]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor


import sys  
sys.path.insert(0, './Python_Models/')
import credentials
# from DataCleaning import clean_text, tokenization, remove_stopwords, listToString, lementization

In [7]:
#punctutation removal
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# Tockenization
def tokenization(text):
    text = re.split('\W+', text)
    return text

# stopword removal 
stopword = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text


# return to string
def listToString(s):     
    # initialize an empty string 
    str1 = " " 
    # return string   
    return (str1.join(s)) 

# Lematizing the words
lemmatizer = WordNetLemmatizer()

def lementization(text):
    text = lemmatizer.lemmatize(text)
    return text


In [8]:
class TwitterAuthenticator():
    def authenticate_twitter_app(self):
        auth = OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
        auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET) 
        return auth       

In [9]:
class TweeterStreamer():
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()
      
    def stream_tweets(self, fetched_tweet_file, hash_tag_list):
        #handles twitter authentication and twitter streaming API
        listener = TwitterListener(fetched_tweet_file) 
        auth = self.twitter_authenticator.authenticate_twitter_app()       
        stream = Stream(auth, listener)
        
        stream.filter(track=hash_tag_list)
         

In [10]:
def DataCleaningFunction(text):
    text = clean_text(text)
    text = tokenization(text.lower())
    text = remove_stopwords(text)
    text = listToString(text)
    text = lementization(text.lower())
    return text


In [11]:
def rounding(results):
    '''Results needs to be rounded to 0 or 1 for fake or real, respectively'''
    if results < 0.5:
        return 0
    else:
        return 1

In [12]:
class TwitterListener(StreamListener):  
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename
    
    def on_data(self, data):
        try:
            tweetText = json.loads(data)['text']
            myvar = tweetText
            tweetText = DataCleaningFunction(tweetText)
            testArr = convert_text_to_index_array(tweetText)
            input = pad_sequences([testArr], padding='post', maxlen=100)
            
            pred = model.predict_generator(input)

            predictions_final = [rounding(x) for x in pred]
            
            print('\n',myvar)
            
            print('****',labels[predictions_final[0]], 'Tweet')
            
            # print("%s \n the %s sentiment; %f%% confidence" % (tweetText ,labels[np.argmax(pred)], pred[0][np.argmax(pred)] * 100))
      
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on data: %s" % str(e))
        return True
    
    def on_error(self, status):
        if status == 420:
            return False
        print(status)
        
        

In [13]:
if __name__ == "__main__":
    mylist =['hazard','fire','ablaze','accident','aftershock','ambulance']   
    fetched_tweets_filename = "./nlp-getting-started/feteched_tweets_text.txt"
    
    tweeter_streamer = TweeterStreamer()
    tweeter_streamer.stream_tweets(fetched_tweets_filename,mylist)
    


 RT @t3ddyyyyy: bom dia crianças
boa ead pra vcs

so nao pode mutar o professor pra jogar free fire ok
&lt;3
**** fake Tweet

 rough day at work super gay did the thing I loathed all day and pretty upset then had a fire evac the building for an hour and it's cold
**** fake Tweet

 RT @FortuneUSNews: 🇺🇸 WHO'S BLOWING 💩 UP IN CHINA?

🇨🇳China simulates blowing up GUAM
🇨🇳Then CHINA has MULTIPLE EXPLOSIONS IN BEIJING
🇨🇳U.S…
**** real Tweet

 Imma hazard a guess and state that Arsenal are done in this summer's transfer window.
**** fake Tweet

 @YoDeZidaneSaes @GolfoMaltes @Elflaco_RM Pagar 100 millones por Hazard acabando contrato y que juegue fuera de form… https://t.co/xI05dakgp4
**** fake Tweet

 RT @mcleeeyn: ice pariss @IcePariss won melody of the year for their song “my ambulance” and ice really deserves this award since the song…
**** fake Tweet

 Navy fires commander of Norfolk Naval Shipyard https://t.co/hIfAl3OinW via @JeffSchogol
**** real Tweet

 RT @SAMOYEDCORE: russia can bec


 RT @mcleeeyn: ice pariss @IcePariss won melody of the year for their song “my ambulance” and ice really deserves this award since the song…
**** fake Tweet

 RT @MazharAbbasGEO: Despite the verdict in Baldia factory fire case after eight long years, the State has failed to fulfil its responsibili…
**** real Tweet

 RT @ChuckRoadsFox46: Accident - I-85 end of ramp to I-77 SB Exit 38 #clttraffic #clt https://t.co/W3wsh5MMjL
**** real Tweet

 RT @AsadHashim: An anti-terrorism court has sentenced two men to death over the 2012 Baldia factory fire, which killed more than 260 people…
**** real Tweet

 RT @Ayoo_SheLOYAL: This fire but who has the time 😩
**** fake Tweet

 RT @KYNDELIATHEGR8: 💔
https://t.co/H2TtHFyvRE
**** fake Tweet

 @SonuSood https://t.co/tuPKPEMkHh
Kindly help my colleague...she is in trouble..🙏🙏🙏
**** fake Tweet

 RT @KASHINNISMF: Wassup twitter. Ion tweet much but i recently got in a life threatening accident. I broke my spine, lost my new car, lost…
**** fake Tweet

 R


 RT @KariJoys: Be fearless in the pursuit of what sets your soul on fire. 

#JoyTrain #SuccessTRAIN #Joy #Success #MentalHealth #Mindfulness…
**** fake Tweet

 J1XTRA: [20:08] NOW PLAYING: Fear and Loathing in Las Vegas - Keep the Heat and Fire Yourself Up ~ 11 listeners  #jpop
**** fake Tweet

 @ConradkBarwa @typerioter Lmfao afaik humans were made out of soil and Djinns out of fire that's why Iblees thought… https://t.co/yGIE0LDXYr
**** fake Tweet

 RT @BBAnimals: mother cat walked through fire FIVE times to save all her kittens. She made a full recovery too https://t.co/h3uRilsMG1
**** fake Tweet

 RT @Whitehouse_P284: Ty Nicolay 2022 2B uncommitted with a nice line drive to RF. Really starting to catch fire. ⁦@JkroegerJt⁩ ⁦@TJHunt_PBR…
**** fake Tweet

 RT @HannibalBateman: @atmosphere When life gives you lemons, you set that shit on fire!
**** fake Tweet

 fire 🔥🔥.
**** fake Tweet

 RT @Kobebay25th: コロナでゲットしたあなたの１０万持って
帰省の交通費は別腹の自腹で
その１０万円をまんま消費したら
観光客とほぼまったく同じ
経済効果があるとは
思わんモンかね？


 RT @naholyr: @LepusReclus Fun fact : le pic d'accident dûs à la collision avec un animal sauvage a lieu… pendant les périodes de chasse.
ht…
**** fake Tweet

 RT @CaseyDancing: casey frey dancing to september by earth, wind &amp; fire https://t.co/PAodGhH5Pl
**** real Tweet

 Don’t forget abt Ericka Plain when the rumors came out she was fired last year, ppl were happy. @Andy &amp; @BravoTV I… https://t.co/4QADNXQVtM
**** fake Tweet

 Earth, Wind &amp; Fire - Fantasy
**** fake Tweet

 RT @KASHINNISMF: Wassup twitter. Ion tweet much but i recently got in a life threatening accident. I broke my spine, lost my new car, lost…
**** fake Tweet

 RT @rahulroushan: Terrorism has no religion, vehicles have
**** real Tweet

 RT @love_at_sundown: #LINETHPeoplesChoiceAwards 

Melody of the Year賞

Ice ParisとPearwahのรักติดไซเรน

Brightが主人公Dr.Pengの青年期を演じたMy Ambulance…
**** fake Tweet

 @elbleda_ Fire Boy Water Girl
**** fake Tweet

 Just had a motorcycle accident. I'm not even hurt. I just feel stu


 RT @_hellothere95: Fan: "Unni this could be a sensitive question but I'm really really curious so I need to ask this, were you kicked out o…
**** fake Tweet

 RT @PattyArquette: You don’t need any more reason to fire Trump than him hiring DeJoy to ruin the US postal service..
**** fake Tweet

 ignore the i at the end it was an accident :D
**** real Tweet

 Praised be You, my Lord, through Brother Fire, through whom You light the night, and he is beautiful and playful an… https://t.co/TFmD6Vbg64
**** fake Tweet

 RT @GLyWDiRLtLuZaae: (拡散希望)動物は愛する大切な家族です。人間一人の力には限界があります。ですが、私達が手を取り合い助け合えば不可能も可能に出来ます。人の為にした事は巡り巡って自分の為になります。尊い命を救う為に、皆様のご支援・拡散お願い致しますm(_…
**** fake Tweet

 RT @FeelingEuphoric: EARTH WIND AND FIRE: do you remember the 21st night of September

ME: no

EARTH WIND AND FIRE *into walkie talkie*: su…
**** real Tweet

 Coral Springs FD Fire Station 64 at sunrise and it's 79.8 F.  Tag image: https://t.co/ZvycIB3hxk https://t.co/LkTe6ICapC
**** real Tweet

 RT @Mariee337: PLEASE ke

KeyboardInterrupt: 